In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
#Variable 
variable85="rcp85_growing_degree_days_gt_4_annual"
variable45="rcp45_growing_degree_days_gt_4_annual"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_growing_degree_days_gt_4_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_growing_degree_days_gt_4_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_growing_degree_days_gt_4_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_growing_degree_days_gt_4_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_growing_degree_days_gt_4_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_growing_degree_days_gt_4_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_growing_degree_days_gt_4_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_growing_degree_days_gt_4_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_growing_degree_days_gt_4_annual.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MP

# FOR RCP 45

## Subsetting arrays by periods of time and percentiles with Xclim

In [4]:
dsEns45= ens.create_ensemble(files45)
dsEns45

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * time                      (time) datetime64[ns] 1950-01-01 ... 2100-01-01
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [5]:
# Mapping data - 30 year means
tmp145 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
window = 30
time145 = tmp145.time[0::window]

ds30yavg45 = tmp145.coarsen(time=window).mean()
ds30yavg45['time'] = time145
perc30yavg45 = ens.ensemble_percentiles(ds30yavg45)
perc30yavg45 # entire grid

# Mapping data

<xarray.Dataset>
Dimensions:                       (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * time                          (time) datetime64[ns] 1981-01-01 ... 2071-01-01
  * lat                           (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                           (lon) float32 -89.04521 ... -54.46326
  * realization                   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_degree_days_gt_4_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_degree_days_gt_4_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [6]:
%time df45 = perc30yavg45.drop('realization').to_dataframe().dropna()

df45

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 2min 29s, sys: 18min 36s, total: 21min 6s
Wall time: 28.7 s


growing_degree_days_gt_4_p10  \
lat       lon        time                                       
66.623306 -89.045212 1981-01-01                    204.300385   
                     2011-01-01                    212.455063   
                     2041-01-01                    261.791809   
                     2071-01-01                    286.726532   
          -88.961884 1981-01-01                    208.835129   
...                                                       ...   
40.041039 -74.129143 2071-01-01                   3754.655762   
          -74.045807 1981-01-01                   3284.001709   
                     2011-01-01                   3573.468506   
                     2041-01-01                   3715.131104   
                     2071-01-01                   3758.138184   

                                 growing_degree_days_gt_4_p50  \
lat       lon        time                                       
66.623306 -89.045212 1981-01-01                    228.999451   
                     2011-01-01                    316.777008   
                     2041-01-01                    362.776215   
                     2071-01-01                    397.703552   
          -88.961884 1981-01-01                    233.610199   
...                                                       ...   
40.041039 -74.129143 2071-01-01                   4121.081055   
          -74.045807 1981-01-01                   3357.985596   
                     2011-01-01                   3613.703125   
                     2041-01-01                   3979.721680   
                     2071-01-01                   4120.909180   

                                 growing_degree_days_gt_4_p90  
lat       lon        time                                      
66.623306 -89.045212 1981-01-01                    245.552460  
                     2011-01-01                    378.937439  
                     2041-01-01                    455.910950  
                     2071-01-01                    561.886353  
          -88.961884 1981-01-01                    250.263245  
...                                                       ...  
40.041039 -74.129143 2071-01-01                   4215.615723  
          -74.045807 1981-01-01                   3372.191650  
                     2011-01-01                   3775.605957  
                     2041-01-01                   4021.009521  
                     2071-01-01                   4215.629883  

[304824 rows x 3 columns]

## Round

In [7]:
df45C = df45.copy()
df45C["growing_degree_days_gt_4_p10"] = round(df45C["growing_degree_days_gt_4_p10"],2)
df45C["growing_degree_days_gt_4_p50"] = round(df45C["growing_degree_days_gt_4_p50"],2)
df45C["growing_degree_days_gt_4_p90"] = round(df45C["growing_degree_days_gt_4_p90"],2)

df45C

growing_degree_days_gt_4_p10  \
lat       lon        time                                       
66.623306 -89.045212 1981-01-01                        204.30   
                     2011-01-01                        212.46   
                     2041-01-01                        261.79   
                     2071-01-01                        286.73   
          -88.961884 1981-01-01                        208.84   
...                                                       ...   
40.041039 -74.129143 2071-01-01                       3754.66   
          -74.045807 1981-01-01                       3284.00   
                     2011-01-01                       3573.47   
                     2041-01-01                       3715.13   
                     2071-01-01                       3758.14   

                                 growing_degree_days_gt_4_p50  \
lat       lon        time                                       
66.623306 -89.045212 1981-01-01                        229.00   
                     2011-01-01                        316.78   
                     2041-01-01                        362.78   
                     2071-01-01                        397.70   
          -88.961884 1981-01-01                        233.61   
...                                                       ...   
40.041039 -74.129143 2071-01-01                       4121.08   
          -74.045807 1981-01-01                       3357.99   
                     2011-01-01                       3613.70   
                     2041-01-01                       3979.72   
                     2071-01-01                       4120.91   

                                 growing_degree_days_gt_4_p90  
lat       lon        time                                      
66.623306 -89.045212 1981-01-01                        245.55  
                     2011-01-01                        378.94  
                     2041-01-01                        455.91  
                     2071-01-01                        561.89  
          -88.961884 1981-01-01                        250.26  
...                                                       ...  
40.041039 -74.129143 2071-01-01                       4215.62  
          -74.045807 1981-01-01                       3372.19  
                     2011-01-01                       3775.61  
                     2041-01-01                       4021.01  
                     2071-01-01                       4215.63  

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [8]:
Region1i45 = df45C.reset_index()
print (Region1i45)
Region1i245 = Region1i45[Region1i45.time.dt.year!= 2011]
print ("2011 eliminated")
df45f = Region1i245.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df45f.columns = [year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df45f

              lat        lon       time  growing_degree_days_gt_4_p10  \
0       66.623306 -89.045212 1981-01-01                        204.30   
1       66.623306 -89.045212 2011-01-01                        212.46   
2       66.623306 -89.045212 2041-01-01                        261.79   
3       66.623306 -89.045212 2071-01-01                        286.73   
4       66.623306 -88.961884 1981-01-01                        208.84   
...           ...        ...        ...                           ...   
304819  40.041039 -74.129143 2071-01-01                       3754.66   
304820  40.041039 -74.045807 1981-01-01                       3284.00   
304821  40.041039 -74.045807 2011-01-01                       3573.47   
304822  40.041039 -74.045807 2041-01-01                       3715.13   
304823  40.041039 -74.045807 2071-01-01                       3758.14   

        growing_degree_days_gt_4_p50  growing_degree_days_gt_4_p90  
0                             229.00                  

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212         3382.55          3697.48          3787.15   
          -88.961884         3378.70          3693.39          3782.72   
          -88.878548         3377.29          3691.69          3780.33   
          -88.795219         3372.48          3686.18          3773.78   
          -88.711891         3378.51          3692.63          3779.04   
...                              ...              ...              ...   
66.623306 -61.629639          132.26           110.18            97.02   
          -61.546310          116.95            95.65            84.21   
          -61.462978          126.15           105.89            93.30   
          -61.379650          128.90           109.60            96.78   
          -61.296322          159.42           141.72           126.86   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212         3438.63          4032.28          4253.47   
          -88.961884         3433.59          4027.83          4249.84   
          -88.878548         3431.03          4025.69          4248.39   
          -88.795219         3425.19          4019.63          4242.81   
          -88.711891         3430.42          4025.38          4249.70   
...                              ...              ...              ...   
66.623306 -61.629639          153.67           156.62           172.81   
          -61.546310          136.32           139.57           152.96   
          -61.462978          147.12           150.12           165.38   
          -61.379650          149.80           153.72           169.35   
          -61.296322          183.81           188.59           209.41   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  
lat       lon                                                           
40.041039 -89.045212         3462.14          4148.50          4306.35  
          -88.961884         3458.41          4143.45          4301.92  
          -88.878548         3456.90          4140.57          4300.01  
          -88.795219         3452.23          4133.68          4294.00  
          -88.711891         3458.78          4138.81          4300.54  
...                              ...              ...              ...  
66.623306 -61.629639          164.56           299.07           264.93  
          -61.546310          147.87           274.91           241.77  
          -61.462978          158.41           289.63           257.43  
          -61.379650          161.44           293.91           262.90  
          -61.296322          194.88           339.61           310.78  

[76206 rows x 9 columns]

# FOR RCP 85

## Subsetting arrays by periods of time and percentiles with Xclim

In [9]:
dsEns85= ens.create_ensemble(files85)
dsEns85

<xarray.Dataset>
Dimensions:                   (lat: 320, lon: 416, realization: 11, time: 151)
Coordinates:
  * time                      (time) datetime64[ns] 1950-01-01 ... 2100-01-01
  * lat                       (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                       (lon) float32 -89.04521 -88.96188 ... -54.46326
  * realization               (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4  (realization, time, lat, lon) float32 dask.array<chunksize=(1, 10, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.

In [10]:
# Mapping data - 30 year means
tmp185 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
window = 30
time185 = tmp185.time[0::window]

ds30yavg85 = tmp185.coarsen(time=window).mean()
ds30yavg85['time'] = time185
perc30yavg85 = ens.ensemble_percentiles(ds30yavg85)
perc30yavg85 # entire grid

<xarray.Dataset>
Dimensions:                       (lat: 320, lon: 416, realization: 11, time: 4)
Coordinates:
  * time                          (time) datetime64[ns] 1981-01-01 ... 2071-01-01
  * lat                           (lat) float32 66.62331 66.53998 ... 40.04104
  * lon                           (lon) float32 -89.04521 ... -54.46326
  * realization                   (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    growing_degree_days_gt_4_p10  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_degree_days_gt_4_p50  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>
    growing_degree_days_gt_4_p90  (time, lat, lon) float32 dask.array<chunksize=(1, 320, 416), meta=np.ndarray>

## Transform into DataFrame

In [11]:
%time df85 = perc30yavg85.drop('realization').to_dataframe().dropna()
df85

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)

  x = np.divide(x1, x2, out)



CPU times: user 1min 41s, sys: 13min 25s, total: 15min 6s
Wall time: 12.7 s


growing_degree_days_gt_4_p10  \
lat       lon        time                                       
66.623306 -89.045212 1981-01-01                    204.170609   
                     2011-01-01                    237.468521   
                     2041-01-01                    299.535278   
                     2071-01-01                    408.056458   
          -88.961884 1981-01-01                    208.370850   
...                                                       ...   
40.041039 -74.129143 2071-01-01                   4342.683594   
          -74.045807 1981-01-01                   3294.425293   
                     2011-01-01                   3603.437988   
                     2041-01-01                   3914.331543   
                     2071-01-01                   4346.732910   

                                 growing_degree_days_gt_4_p50  \
lat       lon        time                                       
66.623306 -89.045212 1981-01-01                    231.743256   
                     2011-01-01                    302.051971   
                     2041-01-01                    408.378296   
                     2071-01-01                    681.121704   
          -88.961884 1981-01-01                    236.792542   
...                                                       ...   
40.041039 -74.129143 2071-01-01                   4853.232422   
          -74.045807 1981-01-01                   3358.894775   
                     2011-01-01                   3713.092773   
                     2041-01-01                   4221.812500   
                     2071-01-01                   4856.269043   

                                 growing_degree_days_gt_4_p90  
lat       lon        time                                      
66.623306 -89.045212 1981-01-01                    243.638214  
                     2011-01-01                    370.006317  
                     2041-01-01                    578.655640  
                     2071-01-01                    890.387573  
          -88.961884 1981-01-01                    248.486786  
...                                                       ...  
40.041039 -74.129143 2071-01-01                   4950.675293  
          -74.045807 1981-01-01                   3373.828613  
                     2011-01-01                   3805.055176  
                     2041-01-01                   4258.879883  
                     2071-01-01                   4953.836914  

[304824 rows x 3 columns]

## Transform in Celsius and round

In [12]:
df85C = df85.copy()
df85C["growing_degree_days_gt_4_p10"] = round(df85C["growing_degree_days_gt_4_p10"],2)
df85C["growing_degree_days_gt_4_p50"] = round(df85C["growing_degree_days_gt_4_p50"],2)
df85C["growing_degree_days_gt_4_p90"] = round(df85C["growing_degree_days_gt_4_p90"],2)

df85C

growing_degree_days_gt_4_p10  \
lat       lon        time                                       
66.623306 -89.045212 1981-01-01                        204.17   
                     2011-01-01                        237.47   
                     2041-01-01                        299.54   
                     2071-01-01                        408.06   
          -88.961884 1981-01-01                        208.37   
...                                                       ...   
40.041039 -74.129143 2071-01-01                       4342.68   
          -74.045807 1981-01-01                       3294.43   
                     2011-01-01                       3603.44   
                     2041-01-01                       3914.33   
                     2071-01-01                       4346.73   

                                 growing_degree_days_gt_4_p50  \
lat       lon        time                                       
66.623306 -89.045212 1981-01-01                        231.74   
                     2011-01-01                        302.05   
                     2041-01-01                        408.38   
                     2071-01-01                        681.12   
          -88.961884 1981-01-01                        236.79   
...                                                       ...   
40.041039 -74.129143 2071-01-01                       4853.23   
          -74.045807 1981-01-01                       3358.89   
                     2011-01-01                       3713.09   
                     2041-01-01                       4221.81   
                     2071-01-01                       4856.27   

                                 growing_degree_days_gt_4_p90  
lat       lon        time                                      
66.623306 -89.045212 1981-01-01                        243.64  
                     2011-01-01                        370.01  
                     2041-01-01                        578.66  
                     2071-01-01                        890.39  
          -88.961884 1981-01-01                        248.49  
...                                                       ...  
40.041039 -74.129143 2071-01-01                       4950.68  
          -74.045807 1981-01-01                       3373.83  
                     2011-01-01                       3805.06  
                     2041-01-01                       4258.88  
                     2071-01-01                       4953.84  

[304824 rows x 3 columns]

## Eliminate 2011, Pivot Table and Create new columns 

In [13]:
Region1i85 = df85C.reset_index()
print (Region1i85)
Region1i285 = Region1i85[Region1i85.time.dt.year!= 2011]
print ("2011 eliminated")
df85f = Region1i285.pivot_table(index=["lat","lon"], columns="time")
print ("DF pivoted")
df85f.columns = [year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
print("Columns renamed")
df85f

              lat        lon       time  growing_degree_days_gt_4_p10  \
0       66.623306 -89.045212 1981-01-01                        204.17   
1       66.623306 -89.045212 2011-01-01                        237.47   
2       66.623306 -89.045212 2041-01-01                        299.54   
3       66.623306 -89.045212 2071-01-01                        408.06   
4       66.623306 -88.961884 1981-01-01                        208.37   
...           ...        ...        ...                           ...   
304819  40.041039 -74.129143 2071-01-01                       4342.68   
304820  40.041039 -74.045807 1981-01-01                       3294.43   
304821  40.041039 -74.045807 2011-01-01                       3603.44   
304822  40.041039 -74.045807 2041-01-01                       3914.33   
304823  40.041039 -74.045807 2071-01-01                       4346.73   

        growing_degree_days_gt_4_p50  growing_degree_days_gt_4_p90  
0                             231.74                  

hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212         3397.20          3945.11          4390.79   
          -88.961884         3393.20          3939.82          4386.10   
          -88.878548         3391.20          3937.01          4383.81   
          -88.795219         3385.85          3931.04          4377.52   
          -88.711891         3391.70          3936.05          4383.63   
...                              ...              ...              ...   
66.623306 -61.629639          145.65           123.03           139.04   
          -61.546310          128.48           109.20           123.70   
          -61.462978          138.98           118.14           133.58   
          -61.379650          142.13           121.03           136.90   
          -61.296322          176.31           150.37           169.28   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212         3424.48          4305.30          4847.89   
          -88.961884         3420.27          4301.73          4844.17   
          -88.878548         3418.46          4300.32          4842.18   
          -88.795219         3413.23          4295.22          4836.22   
          -88.711891         3418.80          4302.28          4842.92   
...                              ...              ...              ...   
66.623306 -61.629639          154.66           185.82           255.05   
          -61.546310          138.56           166.66           226.52   
          -61.462978          148.85           178.29           240.73   
          -61.379650          151.88           182.80           245.56   
          -61.296322          184.82           221.32           303.88   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212         3464.74          4371.49          4999.65  
          -88.961884         3460.41          4366.24          4994.31  
          -88.878548         3458.51          4363.32          4991.64  
          -88.795219         3453.16          4356.16          4984.56  
          -88.711891         3458.69          4360.71          4991.13  
...                              ...              ...              ...  
66.623306 -61.629639          174.74           315.34           468.62  
          -61.546310          157.15           290.59           435.61  
          -61.462978          168.24           308.12           457.36  
          -61.379650          171.60           314.43           463.31  
          -61.296322          207.17           366.04           522.35  

[76206 rows x 9 columns]

# Merge df45 and 85

In [14]:
dftg = pd.merge(df45f, df85f, on=["lat","lon"])
dftg.head()

hist_rcp45_p10  t2050_rcp45_p10  t2080_rcp45_p10  \
lat       lon                                                            
40.041039 -89.045212         3382.55          3697.48          3787.15   
          -88.961884         3378.70          3693.39          3782.72   
          -88.878548         3377.29          3691.69          3780.33   
          -88.795219         3372.48          3686.18          3773.78   
          -88.711891         3378.51          3692.63          3779.04   

                      hist_rcp45_p50  t2050_rcp45_p50  t2080_rcp45_p50  \
lat       lon                                                            
40.041039 -89.045212         3438.63          4032.28          4253.47   
          -88.961884         3433.59          4027.83          4249.84   
          -88.878548         3431.03          4025.69          4248.39   
          -88.795219         3425.19          4019.63          4242.81   
          -88.711891         3430.42          4025.38          4249.70   

                      hist_rcp45_p90  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                            
40.041039 -89.045212         3462.14          4148.50          4306.35   
          -88.961884         3458.41          4143.45          4301.92   
          -88.878548         3456.90          4140.57          4300.01   
          -88.795219         3452.23          4133.68          4294.00   
          -88.711891         3458.78          4138.81          4300.54   

                      hist_rcp85_p10  t2050_rcp85_p10  t2080_rcp85_p10  \
lat       lon                                                            
40.041039 -89.045212         3397.20          3945.11          4390.79   
          -88.961884         3393.20          3939.82          4386.10   
          -88.878548         3391.20          3937.01          4383.81   
          -88.795219         3385.85          3931.04          4377.52   
          -88.711891         3391.70          3936.05          4383.63   

                      hist_rcp85_p50  t2050_rcp85_p50  t2080_rcp85_p50  \
lat       lon                                                            
40.041039 -89.045212         3424.48          4305.30          4847.89   
          -88.961884         3420.27          4301.73          4844.17   
          -88.878548         3418.46          4300.32          4842.18   
          -88.795219         3413.23          4295.22          4836.22   
          -88.711891         3418.80          4302.28          4842.92   

                      hist_rcp85_p90  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                           
40.041039 -89.045212         3464.74          4371.49          4999.65  
          -88.961884         3460.41          4366.24          4994.31  
          -88.878548         3458.51          4363.32          4991.64  
          -88.795219         3453.16          4356.16          4984.56  
          -88.711891         3458.69          4360.71          4991.13

# Extract hist

In [15]:
dftg1 = dftg.drop(columns=["hist_rcp45_p10","hist_rcp45_p50", "hist_rcp45_p90", "hist_rcp85_p10", "hist_rcp85_p50", "hist_rcp85_p90"])
dftgh = dftg[["hist_rcp45_p50", "hist_rcp85_p50"]]

In [16]:
dftg1

t2050_rcp45_p10  t2080_rcp45_p10  t2050_rcp45_p50  \
lat       lon                                                             
40.041039 -89.045212          3697.48          3787.15          4032.28   
          -88.961884          3693.39          3782.72          4027.83   
          -88.878548          3691.69          3780.33          4025.69   
          -88.795219          3686.18          3773.78          4019.63   
          -88.711891          3692.63          3779.04          4025.38   
...                               ...              ...              ...   
66.623306 -61.629639           110.18            97.02           156.62   
          -61.546310            95.65            84.21           139.57   
          -61.462978           105.89            93.30           150.12   
          -61.379650           109.60            96.78           153.72   
          -61.296322           141.72           126.86           188.59   

                      t2080_rcp45_p50  t2050_rcp45_p90  t2080_rcp45_p90  \
lat       lon                                                             
40.041039 -89.045212          4253.47          4148.50          4306.35   
          -88.961884          4249.84          4143.45          4301.92   
          -88.878548          4248.39          4140.57          4300.01   
          -88.795219          4242.81          4133.68          4294.00   
          -88.711891          4249.70          4138.81          4300.54   
...                               ...              ...              ...   
66.623306 -61.629639           172.81           299.07           264.93   
          -61.546310           152.96           274.91           241.77   
          -61.462978           165.38           289.63           257.43   
          -61.379650           169.35           293.91           262.90   
          -61.296322           209.41           339.61           310.78   

                      t2050_rcp85_p10  t2080_rcp85_p10  t2050_rcp85_p50  \
lat       lon                                                             
40.041039 -89.045212          3945.11          4390.79          4305.30   
          -88.961884          3939.82          4386.10          4301.73   
          -88.878548          3937.01          4383.81          4300.32   
          -88.795219          3931.04          4377.52          4295.22   
          -88.711891          3936.05          4383.63          4302.28   
...                               ...              ...              ...   
66.623306 -61.629639           123.03           139.04           185.82   
          -61.546310           109.20           123.70           166.66   
          -61.462978           118.14           133.58           178.29   
          -61.379650           121.03           136.90           182.80   
          -61.296322           150.37           169.28           221.32   

                      t2080_rcp85_p50  t2050_rcp85_p90  t2080_rcp85_p90  
lat       lon                                                            
40.041039 -89.045212          4847.89          4371.49          4999.65  
          -88.961884          4844.17          4366.24          4994.31  
          -88.878548          4842.18          4363.32          4991.64  
          -88.795219          4836.22          4356.16          4984.56  
          -88.711891          4842.92          4360.71          4991.13  
...                               ...              ...              ...  
66.623306 -61.629639           255.05           315.34           468.62  
          -61.546310           226.52           290.59           435.61  
          -61.462978           240.73           308.12           457.36  
          -61.379650           245.56           314.43           463.31  
          -61.296322           303.88           366.04           522.35  

[76206 rows x 12 columns]

# Merge both hist_50 and get the mean

In [17]:
dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)
dftgh2 = dftgh.reset_index()

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dftgh["hist_p50"] = round((dftgh["hist_rcp45_p50"]+ dftgh["hist_rcp85_p50"])/2, 2)



In [18]:
dftgh3 = dftgh2[["lat", "lon", "hist_p50"]]
dftgh3

,lat,lon,hist_p50
0,40.041039,-89.045212,3431.56
1,40.041039,-88.961884,3426.93
2,40.041039,-88.878548,3424.74
3,40.041039,-88.795219,3419.21
4,40.041039,-88.711891,3424.61
...,...,...,...
76201,66.623306,-61.629639,154.16
76202,66.623306,-61.546310,137.44
76203,66.623306,-61.462978,147.99
76204,66.623306,-61.379650,150.84


# Merge means with the 45 and 85 DF

In [19]:
dftg_all = pd.merge(dftgh3, dftg1, on=["lat","lon"])

In [20]:
dftg_all

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,3431.56,3697.48,3787.15,4032.28,4253.47,4148.50,4306.35,3945.11,4390.79,4305.30,4847.89,4371.49,4999.65
1,40.041039,-88.961884,3426.93,3693.39,3782.72,4027.83,4249.84,4143.45,4301.92,3939.82,4386.10,4301.73,4844.17,4366.24,4994.31
2,40.041039,-88.878548,3424.74,3691.69,3780.33,4025.69,4248.39,4140.57,4300.01,3937.01,4383.81,4300.32,4842.18,4363.32,4991.64
3,40.041039,-88.795219,3419.21,3686.18,3773.78,4019.63,4242.81,4133.68,4294.00,3931.04,4377.52,4295.22,4836.22,4356.16,4984.56
4,40.041039,-88.711891,3424.61,3692.63,3779.04,4025.38,4249.70,4138.81,4300.54,3936.05,4383.63,4302.28,4842.92,4360.71,4991.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,154.16,110.18,97.02,156.62,172.81,299.07,264.93,123.03,139.04,185.82,255.05,315.34,468.62
76202,66.623306,-61.546310,137.44,95.65,84.21,139.57,152.96,274.91,241.77,109.20,123.70,166.66,226.52,290.59,435.61
76203,66.623306,-61.462978,147.99,105.89,93.30,150.12,165.38,289.63,257.43,118.14,133.58,178.29,240.73,308.12,457.36
76204,66.623306,-61.379650,150.84,109.60,96.78,153.72,169.35,293.91,262.90,121.03,136.90,182.80,245.56,314.43,463.31


In [21]:
dftg_all.to_feather("growing_degree_days_gt_4_p10_annual.feather")

In [23]:
#dftg_all = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/growing_degree_days_gt_4_p10_annual.feather")

In [24]:
dftg_all 

,lat,lon,hist_p50,t2050_rcp45_p10,t2080_rcp45_p10,t2050_rcp45_p50,t2080_rcp45_p50,t2050_rcp45_p90,t2080_rcp45_p90,t2050_rcp85_p10,t2080_rcp85_p10,t2050_rcp85_p50,t2080_rcp85_p50,t2050_rcp85_p90,t2080_rcp85_p90
0,40.041039,-89.045212,3431.56,3697.48,3787.15,4032.28,4253.47,4148.50,4306.35,3945.11,4390.79,4305.30,4847.89,4371.49,4999.65
1,40.041039,-88.961884,3426.93,3693.39,3782.72,4027.83,4249.84,4143.45,4301.92,3939.82,4386.10,4301.73,4844.17,4366.24,4994.31
2,40.041039,-88.878548,3424.74,3691.69,3780.33,4025.69,4248.39,4140.57,4300.01,3937.01,4383.81,4300.32,4842.18,4363.32,4991.64
3,40.041039,-88.795219,3419.21,3686.18,3773.78,4019.63,4242.81,4133.68,4294.00,3931.04,4377.52,4295.22,4836.22,4356.16,4984.56
4,40.041039,-88.711891,3424.61,3692.63,3779.04,4025.38,4249.70,4138.81,4300.54,3936.05,4383.63,4302.28,4842.92,4360.71,4991.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,154.16,110.18,97.02,156.62,172.81,299.07,264.93,123.03,139.04,185.82,255.05,315.34,468.62
76202,66.623306,-61.546310,137.44,95.65,84.21,139.57,152.96,274.91,241.77,109.20,123.70,166.66,226.52,290.59,435.61
76203,66.623306,-61.462978,147.99,105.89,93.30,150.12,165.38,289.63,257.43,118.14,133.58,178.29,240.73,308.12,457.36
76204,66.623306,-61.379650,150.84,109.60,96.78,153.72,169.35,293.91,262.90,121.03,136.90,182.80,245.56,314.43,463.31


# Merge complete DF with Polygons for each spatial scale

## Dictionary - regioins: column name, short name

In [25]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
              "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"), 
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [26]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [27]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [28]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftg_all, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+replace_all(gdf[name].iloc[0], d)+"_DJC_p10_annual.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_DJC_annual.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_DJC_p10_annual.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
